In [29]:
#KMeans Clustering
import pandas as pd
import numpy as np

data = pd.read_csv('Medical_data.csv')
X = data.values
Y = X[:,1:4]

def distance(a,b):
    d = abs(a-b)
    return np.dot(d,d)**0.5

centers = np.array([[0,0,0],[0,0,0],[0,0,0]])
labels = np.zeros((Y.shape[0]))
count = np.array([Y.shape[0],0,0])

[a,b,c] = np.array([[2,0,0],[0,2,0],[0,0,2]])
tolerance = 10**-2
no_of_iterations = 0

while ((distance(centers[0],a) + distance(centers[1],b) + distance(centers[2],c)) > tolerance) :   
    [p,q,r] = np.zeros((3,3))
    count = np.array([0,0,0])
    for i in range(labels.size):
        d1 = distance(a,Y[i])
        d2 = distance(b,Y[i])
        d3 = distance(c,Y[i])
        m = min(d1,d2,d3)
        if (d1 == m):
            p = p + Y[i]
            labels[i] = 0
            count[0] = count[0] + 1
        elif (d2 == m):
            q = q + Y[i]
            labels[i] = 1
            count[1] = count[1] + 1
        else :
            r = r + Y[i]
            labels[i] = 2
            count[2] = count[2] + 1
    centers = [a,b,c]
    [a,b,c] = (np.array([p,q,r]).T/count).T
    no_of_iterations = no_of_iterations + 1

centers = [a,b,c]
print(no_of_iterations)
print(centers)

c:\users\shiva\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


6
[array([2.457501875913383, 0.25074198298288336, 0.7527665364278691],
      dtype=object), array([0.26840471215934153, 2.5298871818811826, 0.3667307390813902],
      dtype=object), array([0.6850808347844645, 0.11809943924826011, 2.656703189021965],
      dtype=object)]


In [ ]:
#Naive Bayes Classifier
import pandas as pd
import numpy as np

data = pd.read_csv('Medical_data.csv')
X = data.values
Y = X[:,0]
X = X[:,1:4]

dict = {0:"HEALTHY",1:"MEDICATION",2:"SURGERY"}

#distribution(x) - class conditional densities which we find by some method like MLE, Bayesian or anything
unique, counts = np.unique(Y, return_counts=True)
priors = counts/np.sum(counts)

def predict(a):
    i = np.argmax(distribution(a)*priors)
    return dict[i]
def predict_prob(a):
    return (distribution(a)*priors)/np.sum(distribution(a)*priors)

In [18]:
#Gaussian MLE
import pandas as pd
import numpy as np

data = pd.read_csv('Medical_data.csv')
X = data.values
Y = X[:,0]
X = X[:,1:4]

dict = {0:"HEALTHY",1:"MEDICATION",2:"SURGERY"}

mean = []
for i in range(3):
    mean.append(np.mean(X[np.argwhere(Y == dict[i])],axis = 0))
mean = np.asarray(mean)

variance = []
for i in range(3):
    variance.append(np.sum((X[np.argwhere(Y == dict[i])] - mean[i])**2,axis = 0))
variance = np.asarray(variance)

def probability(x,i):
    a = (-np.square(x-mean[i]))/(2*variance[i])
    b = variance[i]**0.5
    p = (1/np.sqrt(2*np.pi))*((np.exp(1)**a)/b)
    return np.prod(p)

def distribution(x):
    d = []
    for i in range(3):
        d.append(probability(x,i))
    d = np.asarray(d)
    return d

print(distribution(np.array([0.211,-0.231,0.99])))

[9.82912493e-06 5.79195789e-06 4.88291949e-06]


In [224]:
#Gaussian Mixture Models
import pandas as pd
import numpy as np

data = pd.read_csv('Medical_data.csv')
X = data.values
Y = X[:,0]
X = X[:,1:4]

def prob_mnd(x,mean,variance): #mulitvariate normal distribution
    a = np.dot(x-mean, np.linalg.inv(variance))
    b = -1/2*np.dot(a, (x-mean))
    d = np.linalg.det(2*np.pi*variance)
    return (d**-0.5)*np.exp(b)

def class_conditionals(X,mean,variance):
    prob_vector = []
    for i in range(np.size(X,0)):
        distribution = []
        for j in range(np.size(mean,0)):
            distribution.append(prob_mnd(X[i],mean[j],variance[j]))
        prob_vector.append(distribution)
    prob_vector = np.asarray(prob_vector)
    return prob_vector

k = 3 # no of gaussians(hyperparameter)

z = np.identity(k)
w = np.tile(z,(int)(Y.size/k) + 1).T
w = w[0:Y.size] 

p = np.mean(w,axis = 0)
mean = []

for i in range(k):
    mean.append(np.mean(X*np.reshape(w.T[i],(Y.size,1)), axis = 0)/(p[i]))
mean = np.asarray(mean, dtype = float)

variance = []
for i in range(k):
    temp = 0
    for j in range(Y.size):
        temp += w[j][i]*(np.asmatrix(X[i])).T*np.asmatrix(X[i])
    variance.append(temp/(Y.size*p[i]))
variance = np.asarray(variance, dtype = float)


w_dash = class_conditionals(X,mean,variance)*p
w_dash = w_dash/np.reshape(np.sum(w_dash,axis = 1),(Y.size,1))

iterations = 0
tolerance = 0.00001

while np.linalg.norm(w_dash - w)>tolerance and iterations<1000 :
    w = w_dash
    
    p = np.mean(w,axis = 0)
    mean = []
    for i in range(k):
        mean.append(np.sum(X*np.reshape(w.T[i],(Y.size,1)))/(Y.size*p[i]))
    mean = np.asarray(mean)
    variance = []
    for i in range(k):
        temp = 0
        for j in range(Y.size):
            temp += w[j][i]*(np.asmatrix(X[i])).T*np.asmatrix(X[i])
        variance.append(temp/(Y.size*p[i]))
    variance = np.asarray(variance,dtype = float)
    
    w_dash = class_conditionals(X,mean,variance)*p
    w_dash = w_dash/np.reshape(np.sum(w_dash,axis = 1),(Y.size,1))
w = w_dash


c:\users\shiva\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]
c:\users\shiva\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in exp
  del sys.path[0]
